# Introduction

This tutorial demonstrates the most important basic steps involved in the analysis of scanning electron diffraction data.

The data was acquired from a GaAs nanowire adopting the zinc blende structure and exhibiting type I twinning (i.e. on {111}) along its length. The region of interest contains a single nanowire comprising multiple crystals each in one of the two twinned orientations and near to a <1-10> zone axis.

This functionaility has been checked to run in pyxem version 0.13.1 (April 2021). However, bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/pyxem-demos/issues

# Contents

1. <a href='#loa'> Loading & Inspection</a>
2. <a href='#cal'> Alignment & Calibration</a>
3. <a href='#vdf'> Virtual Diffraction Imaging</a>
4. <a href='#ml'> Machine Learning SPED Data</a>
5. <a href='#vec'> Peak Finding</a>

Import pyxem and other required libraries

In [1]:
%matplotlib tk
import hyperspy.api as hs
import pyxem as pxm
import numpy as np

# <a id='loa'></a> 1. Loading and Inspection

Load the SPED data acquired from the nanowire

In [3]:
dp = hs.load('./data/01/twinned_nanowire.hdf5', reader='hspy')

C:\Users\emilc\AppData\Local\miniforge3\envs\TEM-intro-2024\Lib\site-packages\hyperspy\io.py:650: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn(


Inspect the dp object

In [4]:
dp

<Diffraction2D, title: , dimensions: (30, 100|144, 144)>

Specify that the data is electron diffraction data

In [5]:
dp.set_signal_type('electron_diffraction')

Inspect the signal type

In [6]:
dp

<ElectronDiffraction2D, title: , dimensions: (30, 100|144, 144)>

Inspect the data type of the object

In [7]:
dp.data.dtype

dtype('uint8')

Inspect the metadata associated with the object 'dp'

In [8]:
dp.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── beam_energy = 300.0
│       ├── camera_length = 0.21000000000000002
│       └── scan_rotation = 277.0
├── General
│   ├── FileIO
│   │   └── 0
│   │       ├── hyperspy_version = 2.1.1
│   │       ├── io_plugin = rsciio.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2024-09-04T07:54:04.118668+02:00
│   ├── original_filename = nanowire_precession.blo
│   ├── time = (2014, 12, 8)
│   └── title = 
└── Signal
    ├── signal_origin = 
    └── signal_type = electron_diffraction

Set important experimental parameters using the built in function

In [9]:
dp.set_experimental_parameters(beam_energy=300.0,
                               camera_length=21.0,
                               scan_rotation=277.0,
                               convergence_angle=0.7,
                               exposure_time=10.0)

See how this changed the metadata

In [10]:
dp.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       ├── camera_length = 21.0
│       │       └── exposure_time = 10.0
│       ├── beam_energy = 300.0
│       ├── camera_length = 0.21000000000000002
│       ├── convergence_angle = 0.7
│       └── scan_rotation = 277.0
├── General
│   ├── FileIO
│   │   └── 0
│   │       ├── hyperspy_version = 2.1.1
│   │       ├── io_plugin = rsciio.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2024-09-04T07:54:04.118668+02:00
│   ├── original_filename = nanowire_precession.blo
│   ├── time = (2014, 12, 8)
│   └── title = 
└── Signal
    ├── signal_origin = 
    └── signal_type = electron_diffraction

Set another metadata item and check it

In [11]:
dp.metadata.set_item("General.title", 'GaAs Nanowire')
dp.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       ├── camera_length = 21.0
│       │       └── exposure_time = 10.0
│       ├── beam_energy = 300.0
│       ├── camera_length = 0.21000000000000002
│       ├── convergence_angle = 0.7
│       └── scan_rotation = 277.0
├── General
│   ├── FileIO
│   │   └── 0
│   │       ├── hyperspy_version = 2.1.1
│   │       ├── io_plugin = rsciio.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2024-09-04T07:54:04.118668+02:00
│   ├── original_filename = nanowire_precession.blo
│   ├── time = (2014, 12, 8)
│   └── title = GaAs Nanowire
└── Signal
    ├── signal_origin = 
    └── signal_type = electron_diffraction

In [12]:
dp

<ElectronDiffraction2D, title: GaAs Nanowire, dimensions: (30, 100|144, 144)>

Plot the data to inspect it

In [13]:
dp.plot(cmap='inferno', vmax=50)

# <a id='cal'></a> 2. Alignment & Calibration

Apply distortion corrections to the data due to off-axis acquisition

In [14]:
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]),
                               keep_dtype=True)

[########################################] | 100% Completed | 1.73 ss


In [15]:
dp.data.dtype

dtype('uint8')

Align the dataset based on the direct beam position

In [16]:
dp.center_direct_beam(method='cross_correlate',
                      radius_start=2,
                      radius_finish=5,
                      half_square_width=10)

[########################################] | 100% Completed | 6.46 ss
[########################################] | 100% Completed | 1.32 ss


In [17]:
dp.plot(cmap='inferno', vmax=50)

Measure known interplanar spacing to obtain calibration

In [18]:
dpm = dp.mean((0,1))
dpm.plot(cmap='inferno', vmax=50)
line = hs.roi.Line2DROI(x1=25.8525, y1=64.5691, x2=120.907, y2=77.0079, linewidth=5.49734)
line.add_widget(dpm)

In [19]:
trace = line(dpm)
trace.plot()

In [20]:
recip_d111 = np.sqrt((3/5.6535**2))
recip_cal = recip_d111 / 11.4

Set data calibrations

In [21]:
dp.set_diffraction_calibration(recip_cal)
dp.set_scan_calibration(10)

Plot the calibrated data

In [22]:
dp.plot(cmap='inferno', vmax=50)

# <a id='vdf'></a> 3. Virtual Diffraction Imaging & Selecting Regions

## 3.1 Interactive VDF Imaging

Plot an interactive virtual image integrating intensity within a circular subset of pixels in the diffraction pattern

In [23]:
roi = hs.roi.CircleROI(cx=0.,cy=0, r_inner=0, r=0.07)

In [24]:
dp.plot_integrated_intensity(roi)

Get the virtual diffraction image associated with the last integration window used interactively

In [25]:
vdf = dp.get_integrated_intensity(roi)

Plot the virtual dark-field image

In [26]:
vdf.plot()

Inspect the metadata

In [27]:
vdf.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       ├── camera_length = 21.0
│       │       └── exposure_time = 10.0
│       ├── beam_energy = 300.0
│       ├── camera_length = 0.21000000000000002
│       ├── convergence_angle = 0.7
│       └── scan_rotation = 277.0
├── Diffraction
│   └── integrated_range = CircleROI(cx=-0.483739, cy=0.134372, r=0.0671859, r_inner=0) of GaAs Nanowire
├── General
│   ├── FileIO
│   │   └── 0
│   │       ├── hyperspy_version = 2.1.1
│   │       ├── io_plugin = rsciio.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2024-09-04T07:54:04.118668+02:00
│   ├── original_filename = nanowire_precession.blo
│   ├── time = (2014, 12, 8)
│   └── title = Integrated intensity
└── Signal
    ├── signal_origin = 
    └── signal_type =

Save the virtual dark-field image as a 32bit tif

In [28]:
vdf.change_dtype('float32')
vdf.save('vdfeg.tif')

## 3.2 Form multiple images using a VDF Generator

Import the VirtualImageGenerator class

In [29]:
from pyxem.generators.virtual_image_generator import VirtualImageGenerator

Initialize the VDFGenerator

In [30]:
vdfgen = VirtualImageGenerator(dp)

Calculate 10 annular VDF images between 0 and 1 reciprocal angstroms

In [31]:
vdfs = vdfgen.get_concentric_virtual_images(k_min=0,
                                        k_max=1,
                                        k_steps=10)

C:\Users\emilc\AppData\Local\miniforge3\envs\TEM-intro-2024\Lib\site-packages\pyxem\generators\virtual_image_generator.py:58: VisibleDeprecationWarning: Function `get_concentric_virtual_images()` is deprecated and will be removed in version 1.0.0. Use `pyxem.signals.Diffraction2D.get_azithumal_integral1d()` instead.
  alternative="pyxem.signals.Diffraction2D.get_azithumal_integral1d",


Plot the VDF images

In [32]:
vdfs.plot()

Save the stack of VDF image as a 32bit tif stack

In [ ]:
vdfs.change_dtype('float32')
vdfs.save('vdsfeg.tif')

## 3.3 Select a region in the scan

Plot the data with an adjustable marker indicating where to crop the scan region

In [ ]:
reg = hs.roi.RectangularROI(left=50, top=630, right=290, bottom=870)
dp.plot(cmap='inferno', vmax=50)
reg.add_widget(dp)

Crop the dataset based on the region defined above

In [ ]:
dpc = reg(dp)

Calculate the mean diffraction pattern from the selected region

In [ ]:
dpcm = dpc.mean((0,1))

Plot the mean diffraction pattern from the selected region

In [ ]:
dpcm.plot(cmap='inferno', vmax=50)

# <a id='ml'></a> 4. Unsupervised learning

Perform singular value decomposition (SVD) of the data

In [ ]:
dpc.data = dpc.data.astype('float64')
dpc.decomposition(True, algorithm='SVD')

Obtain a "Scree plot" by plotting the fraction of variance described by each principal component

In [ ]:
dpc.plot_explained_variance_ratio()

Plot the decomposition results and have a look at them

In [ ]:
dpc.plot_decomposition_results()

Perform non-negative matrix factorisation (NMF)

In [ ]:
dpc.decomposition(True, algorithm='NMF', output_dimension=3)

Plot the NMF results

In [ ]:
dpc.plot_decomposition_results()

# <a id='vec'></a> 5. Peak Finding

Perform peak finding on all diffraction patterns in data

In [ ]:
peaks = dpc.find_peaks(method='difference_of_gaussian',
                       min_sigma=1.,
                       max_sigma=6.,
                       sigma_ratio=1.6,
                       threshold=0.04,
                       overlap=0.99,
                       interactive=False)

Check the peaks object type

In [ ]:
peaks

Look at what's in the peaks object

In [ ]:
print(peaks.inav[0,0].data)
print(" ")
print(peaks.inav[1,1].data)

coaxing peaks back into a DiffractionVectors

In [ ]:
from pyxem.signals.diffraction_vectors import DiffractionVectors

In [ ]:
peaks = DiffractionVectors.from_peaks(peaks,center=(72,72),calibration=recip_cal)

Plot found peak positions as an overlay on the data

In [ ]:
peaks.plot_diffraction_vectors_on_signal(dpc, cmap='gray', vmax=50)

Form a diffracting pixels map to show where peaks were found and plot it

In [ ]:
crystim = peaks.get_diffracting_pixels_map(binary=False)
crystim.plot(cmap='viridis')